In [1]:
import os
import pandas as pd
import cx_Oracle
from dotenv import load_dotenv

In [12]:
load_dotenv()
username = os.getenv('ORACLE_USERNAME')
password = os.getenv('ORACLE_PASSWORD')
dsn = os.getenv('ORACLE_DSN')
pool = cx_Oracle.SessionPool(user=username, password=password, dsn=dsn, min=2, max=5, increment=1)

In [9]:
KID='397451/2024'
AHT='397451'

In [11]:
df = pd.read_excel('./data/BETÖLTŐ_május.xlsx', usecols=[1, 2, 6], names=['KSH', 'ONK', 'AMOUNT'])
df['KSH'] = df['KSH'].astype(str).str.zfill(7)
df['KID'] = KID
df['AHT'] = AHT

df.head()

,KSH,ONK,AMOUNT,KID,AHT
0,1904561,Ábrahámhegy,720000,397451/2024,397451
1,1406080,Ádánd,750000,397451/2024,397451
2,0303522,Baja,300000,397451/2024,397451
3,0402680,Békésszentandrás,1050000,397451/2024,397451
4,1520677,Beregsurány,765000,397451/2024,397451


In [13]:
#connection = pool.acquire()
for index, row in df.iterrows():
    ksh_value = row['KSH']
    print(ksh_value)
#pool.release(connection)

1904561
1406080
0303522
0402680
1520677
1308891
1706497
0113578
0118069
0108208
0326471
1530641
1613170
1707685
1125131
1616647
0314058
0202857
2002097
0405032
0912803
1907898
0511697
0608314
1213204
1618209
0828334
0322530
1117330
0522840
0320297
0332434
2016081
2016364
0527395
0507825
0718485
0828149
1221102
1826143
0423065
0207010
2005704
1033686
0517048
0833701
0531884
0723694
1108688
0614456
1831583
0216355
1011013
1803009
0923214
0819035
1630386
1628228
1509919
1318342
1911767


In [14]:
pool.close()